## Testing how to vectorize the labels to polygons (list of points)
Using skimage.measure.regionprops + cv2.findContours seems to be the best way.

In [ ]:
from skimage import measure
import numpy as np
import IPython
import cv2 as cv2
import os

def imshow(img,size=(500,500)):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret,width=size[0],height=size[1])
    IPython.display.display(i)

def flatten(l):
    return [item for sublist in l for item in sublist]

img_name = "cubemap_P74_0030931c0e3aaaa3a10a9e0f63195eb2_id_negx"

image = cv2.imread(f"SISTEMAS_UFF/Labels/{img_name}.png")[:,:,0]

image[image<255] += 1
#image[image<255] *= 20

#image[700:800,1000:1100] = 100

width = image.shape[0]
height = image.shape[1]

print(np.unique(image))
print(image.shape)
print(image.shape)


copied = np.copy(image)
regions = measure.regionprops(copied)
print(f"number of classes present: {len(regions)}")

obj_number = 0

imshow(image)
canvas_for_polygon = np.zeros_like(image)+255
yolo_obj_file = ""

for class_id in range(len(regions)):
    class_present = regions[class_id]
    canvas_for_mask = np.zeros_like(image)
    if class_present.label<255:

        print(f"class_label: {class_present.label}")
        bounding_boxes = class_present.bbox
        print(f"bounding_boxes: {bounding_boxes}")
        BB_index = np.s_[bounding_boxes[0]:bounding_boxes[2], bounding_boxes[1]:bounding_boxes[3]]
        
        canvas_for_mask[BB_index] = class_present.image

        print((class_present.image*255).shape)

        contours, _ = cv2.findContours((canvas_for_mask).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        print(f"type(contours): {type(contours)}")
        print(f"len(contours): {len(contours)}")
        obj_number += len(contours)
        #print(f"(contours): {(contours)}")
        for object_ in contours:
            point_list = []
            for point in object_:
                x = point[0][0]/width
                y = point[0][1]/height
                point_list.append((x,y))
            
            
            points = [str(item) for item in flatten(point_list)]
            yolo_obj_file += f"{class_present.label-1} {' '.join(points)}\n"
            #print(f"YOLO label:{yolo_obj_line}")
            
        
        
        canvas_for_polygon = cv2.fillPoly(canvas_for_polygon, contours, class_present.label)
        #canvas_for_polygon = cv2.fillPoly(canvas_for_polygon, [contours[5]], class_present.label)
        print(f"")

print(f"number of objects: {obj_number}")
img_name = img_name[:-7]+img_name[-4:]

if os.path.exists("/home/pguedes/YOLOv8/Darwin_dados/SISTEMAS_UFF/Labels/output"):
    delete_all_files_in_folder("/home/pguedes/YOLOv8/Darwin_dados/SISTEMAS_UFF/Labels/output")
else:
    os.makedirs("/home/pguedes/YOLOv8/Darwin_dados/SISTEMAS_UFF/Labels/output")
    
with open(f"/home/pguedes/YOLOv8/Darwin_dados/SISTEMAS_UFF/Labels/output/{img_name}.txt", "w+") as f:
    for yolo_obj_line in yolo_obj_file:
        f.write(yolo_obj_line)
        
imshow((canvas_for_polygon))
print((canvas_for_polygon-image).sum())

In [ ]:
from multiprocessing import Pool
from utils import try_find_file
from tqdm.notebook import tqdm
from shapely import Polygon
from skimage import measure
import numpy as np
import IPython
import cv2
import os

NUM_THREADS = max(1, os.cpu_count() - 1)

def delete_all_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                delete_all_files_in_folder(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
            
            
def flatten(l):
    return [item for sublist in l for item in sublist]

def vectorize_label_file(label_path:str, base_directory="SISTEMAS_UFF/Labels/",output_path="SISTEMAS_UFF/Labels/output", verbose=False):
    
    file_found = try_find_file(label_path, base_directory)
    if file_found == 0:
        raise ValueError(f"file {label_path} not found")
        return
    elif file_found == 1:
        file_path = label_path
    elif type(file_found) == str:
        file_path = file_found
        
    img_name = os.path.split(file_path)[-1]
    

    image = cv2.imread(file_path)
    width = image.shape[0]
    height = image.shape[1]
    

    if image.ndim > 3 or image.ndim < 2:
        raise ValueError(f"only 2 or 3 dimensions are supported")
        return
    if image.ndim == 3:
        if verbose:
            print(f"Image {file_path} seems to be RGB, converting to Grayscale")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    print(f"Converting {img_name}... ",end="")

    image[image<255] += 1

    copied = np.copy(image)
    regions = measure.regionprops(copied)
    
    obj_number = 0

    if verbose:
        print(np.unique(image))
        print(image.shape)
        print(len(regions))

    # create a white canvas to draw on
    canvas_for_polygon = np.zeros_like(image)+255
    
    yolo_obj_file = ""
    
    for class_id in range(len(regions)):
        class_present = regions[class_id]
        canvas_for_mask = np.zeros_like(image)

        if class_present.label<255:
            bounding_boxes = class_present.bbox
            BB_index = np.s_[bounding_boxes[0]:bounding_boxes[2], bounding_boxes[1]:bounding_boxes[3]]
            
            canvas_for_mask[BB_index] = class_present.image


            contours, _ = cv2.findContours((canvas_for_mask).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            
            obj_number += len(contours)
            #print(f"(contours): {(contours)}")
            for object_ in contours:
                point_list = []
                for point in object_:
                    x = point[0][0]/width
                    y = point[0][1]/height
                    point_list.append((x,y))
                
                
                points = [str(item) for item in flatten(point_list)]
                yolo_obj_file += f"{class_present.label-1} {' '.join(points)}\n"
                
            
            if verbose:
                print(f"class_label: {class_present.label}")
                print(f"bounding_boxes: {bounding_boxes}")
                print((class_present.image*255).shape)
                print(f"type(contours): {type(contours)}")
                print(f"len(contours): {len(contours)}")
                print(f"")
    if verbose:
        print(f"number of objects: {obj_number}")
        
    img_name = img_name.replace("_id","").replace(".png","")
    
    

    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    with open(f"{output_path}/{img_name}.txt", "w+") as f:
        for yolo_obj_line in yolo_obj_file:
            f.write(yolo_obj_line)
    
    print(f"Done!")
        
output_path = "SISTEMAS_UFF/Labels/output"
    
if os.path.exists(output_path):
    delete_all_files_in_folder(output_path)
else:
    os.makedirs(output_path)


pool = Pool(NUM_THREADS)

failed_list = []

#vectorize_label_file("cubemap_P74_0030931c0e3aaaa3a10a9e0f63195eb2_id_negx.png")
for entries in os.listdir("SISTEMAS_UFF/Labels/"):
    if os.path.isfile(f"SISTEMAS_UFF/Labels/{entries}"):
        #vectorize_label_file(entries)
        pool.apply_async(vectorize_label_file, args=(entries,))
    else:
        failed_list.append(entries)
        
pool.close()
pool.join()

print(failed_list)
    